In [ ]:
import tensorflow as tf 
import matplotlib as plt
from sklearn.cluster import KMeans
import numpy as np
import cv2
import os
import glob
import shutil

In [ ]:
# Run this cell if you have issues with intitializing cuDNN
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

In [ ]:
input_dir = 'pets'
output_dir = input_dir + '/output'
glob_dir = input_dir + '/*.jpg'

In [ ]:
images = [cv2.cvtColor(cv2.resize(cv2.imread(file), (224, 224)),cv2.COLOR_BGR2RGB) for file in glob.glob(glob_dir)]
paths = [file for file in glob.glob(glob_dir)]
images = np.array(np.float32(images).reshape(len(images), -1)/255)

In [ ]:
model = tf.keras.applications.InceptionV3(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
predictions = model.predict(images.reshape(-1, 224, 224, 3))
pred_images = predictions.reshape(images.shape[0], -1)

In [ ]:
kmodel = KMeans(n_clusters=2, n_jobs=-1, random_state=728)
kmodel.fit(pred_images)
kpredictions = kmodel.predict(pred_images)
shutil.rmtree('output')
for i in range(2):
	os.makedirs("output\cluster" + str(i))
for i in range(len(paths)):
	shutil.copy2(paths[i], "output\cluster"+str(kpredictions[i]))